<a href="https://colab.research.google.com/github/ishikawa238/ishikawa_program/blob/main/Multi_TaskDNN%E3%81%BE%E3%81%A8%E3%82%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前処理

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gpytorch
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyl

In [3]:
import gpytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from gpytorch.models import ApproximateGP
# from gpytorch.variational import (CholeskyVariationalDistribution, WhitenedVariationalStrategy)

from torch.nn import GRU, Linear, ReLU, Sequential
from torch_geometric.data import DataLoader
from torch_geometric.datasets import QM9
from torch_geometric.nn import NNConv, Set2Set
from torch_geometric.utils import remove_self_loops


# データ準備

## Data_Loader

In [4]:
import numpy as np
import math
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.utils import losses_utils
import tensorflow_probability as tfp
from tqdm import tqdm


tf.random.set_seed(1234)

bs = 32
class Data_Loader(object):
    def __init__(self, separate_char, name="data"):
        self.separate_char = separate_char
        self.mean_list = []
        self.std_list = []
    def normalize(self,input_data):
      # epsilon=0
      m = np.mean(input_data, keepdims=True)
      std = np.std(input_data, keepdims=True)
      X_normed = (input_data - m) / std

      return X_normed,m,std


    def normalize_valid(self,input_data,num):
      epsilon=0.001
      X_normed = (input_data - self.mean_list[num]) / math.sqrt(self.std_list[num]**2 + epsilon)
      return X_normed

    def min_max_normalize(self,data, min, max):
      # min_val = min(data)
      # max_val = max(data)
      # 復元しやすくするために固定
      min_val = min
      max_val = max
      normalized = [(x - min_val) / (max_val - min_val) for x in data]
      return normalized, min, max

    def load_data(self, path,mode, itemSize):
        f_data = open(path, 'r')
        item_vector_data = []
        correct_data = []
        RT_data = []
        RT_data_fission = []
        theta_data = []

        # f_dataの整理
        for lineID, line in enumerate(f_data):
          if lineID == 0:
            pass
          else:
            # l通过行号来读取数据
            line = line.split(self.separate_char)

            # itmevectorだけいっぺんにいれる
            item_list = []
            # itemsize = 134
            itemsize = itemSize
            # for j in range(100):
            for j in range(itemsize):
              item_list.append(float(line[j+2]))

            item_vector_data.append(item_list)
            correct_data.append(float(line[itemsize+2]))
            RT_data.append(float(line[itemsize+3]))
            RT_data_fission.append(float(line[itemsize+3]))
            theta_data.append(float(line[itemsize+4]))

        f_data.close()

        ### data: [[],[],[],...] <-- set_max_seqlen is used
        # convert data into ndarrays for better speed during training
        if mode =="train":
          RT_dataArray, min, max = self.min_max_normalize(np.array(RT_data), 0, 1000)
          # RT_dataArray= self.normalize_valid(np.array(RT_data), 0)
          # theta_dataArray = self.normalize_valid(np.array(theta_data), 0
          theta_dataArray, min2, max2 = self.min_max_normalize(np.array(theta_data), -3, 3)

          self.mean_list = min
          self.std_list = max
        if mode == "valid":
          # RT_dataArray= self.normalize_valid(np.array(RT_data), 0)
          # theta_dataArray = self.normalize_valid(np.array(theta_data), 0)
          RT_dataArray, min, max = self.min_max_normalize(np.array(RT_data), 0, 1000)
          theta_dataArray, min2, max2 = self.min_max_normalize(np.array(theta_data), -3, 3)


        if mode !="train":
          return item_vector_data, correct_data, RT_dataArray, theta_dataArray
          # return charge_number_dataArray, charge_number_fission_dataArray, energy_dataArray, atomic_number_dataArray,fission_yield_dataArray,fission_yield_nomalize_dataArray,error_value_dataArray,odd_even_dataArray,sum(fission_yield_dataArray)/len(fission_yield_dataArray)
        else:
          return item_vector_data, correct_data, RT_dataArray, theta_dataArray
          # return charge_number_dataArray, charge_number_fission_dataArray, energy_dataArray, atomic_number_dataArray,fission_yield_dataArray,fission_yield_nomalize_dataArray,error_value_dataArray,odd_even_dataArray

## データを読み込み


In [5]:
dat = Data_Loader(separate_char=',')

# 訓練データ読み込み(とりあえず全部)
# train_data_path = '/content/drive/MyDrive/dataset/train_df2.csv'
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/new_df_theta.csv'
# valid_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_theta_categorize_0.05-1.csv'
# test_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_theta_categorize_0.05-1.csv'

# 物理
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/df_physics_theta_LNIRT_overt0.csv'
# valid_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_physics_UEC2023.csv'
# test_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_physics_UEC2023.csv'

# 情報
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/df_information_theta_LNIRT_overt0.csv'
# valid_data_path = '/content/drive/MyDrive/MMOE/dataset/test_information_overt0_LNIRT.csv'
# test_data_path = '/content/drive/MyDrive/MMOE/dataset/test_information_overt0_LNIRT.csv'

# UEC2024　物理
train_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/物理_trainData_overt0_theta-LNIRT.csv'
valid_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_物理.csv'
test_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_物理.csv'

# UEC2024　数学
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/数学_trainData_overt0.csv'
# valid_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_数学.csv'
# test_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_数学.csv'


# itemの数
# itemsize = 100
# itemsize = 134
# itemsize = 284
itemsize = 238
# itemsize=150

# 時間データのみ正規化?
train_itemvec, train_response, train_time, train_theta = dat.load_data(train_data_path,mode = "train", itemSize=itemsize)
# print(dat.mean_list, dat.std_list)
print(train_theta)
train_norm_list = [dat.mean_list, dat.std_list]
print(len(train_norm_list), train_norm_list)
valid_itemvec, valid_response, valid_time, valid_theta = dat.load_data(valid_data_path,mode = "valid", itemSize=itemsize)
test_itemvec, test_response, test_time, test_theta = dat.load_data(test_data_path,mode = "train", itemSize=itemsize)
print(dat.mean_list, dat.std_list)
test_norm_list = [dat.mean_list] + [dat.std_list]
# print(len(train_itemvec))
# print(train_response)
# print(train_time)


label_keys = [f'item{i}' for i in range(itemsize)] + ["response", "time", "student_theta"]
# print(label_keys)
label_values_train = [np.array(train_itemvec).T[i] for i in range(itemsize)] + [train_response, train_time, train_theta]
label_values_valid = [np.array(valid_itemvec).T[i] for i in range(itemsize)] + [valid_response, valid_time, valid_theta]
label_values_test = [np.array(test_itemvec).T[i] for i in range(itemsize)] + [test_response, test_time, test_theta]
# print(label_values)
# ラベルを辞書に変換
data_dict_train = dict(zip(label_keys, label_values_train))
data_dict_valid = dict(zip(label_keys, label_values_valid))
data_dict_test = dict(zip(label_keys, label_values_test))
# print(data_dict_train["time"])


[0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5742588922146511, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721, 0.5370363633924721,

In [6]:
from torch.utils.data import Dataset, DataLoader, Subset
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


label_keys = [f'item{i}' for i in range(itemsize)]
label_target = ["response", "time"]
# print(label_keys)
label_keys_values_train = np.array([np.array(train_itemvec).T[i] for i in range(itemsize)]+[train_theta]).T
label_target_values_train = np.array([train_response, train_time]).T

# test
label_keys_values_test = np.array([np.array(test_itemvec).T[i] for i in range(itemsize)]+[test_theta]).T
label_target_values_test = np.array([test_response, test_time]).T

# ラベルを辞書に変換
X_train = label_keys_values_train # dict(zip(label_keys, label_keys_values_train))
y_train = label_target_values_train #dict(zip(label_target, label_target_values_train))
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# testも同様に変換
X_test = label_keys_values_test # dict(zip(label_keys, label_keys_values_test))
y_test = label_target_values_test #dict(zip(label_target, label_target_values_test))
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

#deep Gaussian process 使用例

In [8]:
import sys
import os

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

!pip install gpytorch
import gpytorch
from gpytorch.means import ConstantMean, LinearMean
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.variational import VariationalStrategy, CholeskyVariationalDistribution
from gpytorch.distributions import MultivariateNormal
from gpytorch.models import ApproximateGP, GP
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.likelihoods import GaussianLikelihood

from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL


In [9]:
class ToyDeepGPHiddenLayer(DeepGPLayer):
    def __init__(self, input_dims, output_dims, num_inducing=128, mean_type='constant'):
        if output_dims is None:
            inducing_points = torch.randn(num_inducing, input_dims)
            batch_shape = torch.Size([])
        else:
            inducing_points = torch.randn(output_dims, num_inducing, input_dims)
            batch_shape = torch.Size([output_dims])

        variational_distribution = CholeskyVariationalDistribution(
            num_inducing_points=num_inducing,
            batch_shape=batch_shape
        )

        variational_strategy = VariationalStrategy(
            self,
            inducing_points,
            variational_distribution,
            learn_inducing_locations=True
        )

        super(ToyDeepGPHiddenLayer, self).__init__(variational_strategy, input_dims, output_dims)

        if mean_type == 'constant':
            self.mean_module = ConstantMean(batch_shape=batch_shape)
        else:
            self.mean_module = LinearMean(input_dims)
        self.covar_module = ScaleKernel(
            RBFKernel(batch_shape=batch_shape, ard_num_dims=input_dims),
            batch_shape=batch_shape, ard_num_dims=None
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

class DeepGPModel(DeepGP):
    def __init__(self, input_dim=100):
        super().__init__()
        num_hidden_dims = 10

        self.input_layer = ToyDeepGPHiddenLayer(
            input_dims=input_dim,
            output_dims=num_hidden_dims,
            mean_type='linear',
        )

        self.hidden_layer = ToyDeepGPHiddenLayer(
            input_dims=num_hidden_dims,
            output_dims=num_hidden_dims,
            mean_type='linear',
        )

        self.last_layer = ToyDeepGPHiddenLayer(
            input_dims=num_hidden_dims,
            output_dims=None,
            mean_type='constant',
        )

        self.likelihood = GaussianLikelihood()

    def forward(self, inputs):
        h = self.input_layer(inputs)
        h = self.hidden_layer(h)
        output = self.last_layer(h)
        return output

    def predict(self, x):
        preds = self.likelihood(self.forward(x))
        return preds.mean, preds.variance



# Multi-Task DNN

## モデル

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ExpertNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = F.leaky_relu(layer(x), negative_slope=0.2)
        return x

class BayesianNuclearDeep(nn.Module):
    def __init__(self, input_dim, hidden_dim, experts, num_layers):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)

        self.experts = nn.ModuleList([ExpertNetwork(hidden_dim, hidden_dim, num_layers) for _ in range(experts)])

        self.task1_gate_layer = nn.Linear(hidden_dim, experts)
        self.task2_gate_layer = nn.Linear(hidden_dim, experts)

        self.output1_layer = ExpertNetwork(hidden_dim, hidden_dim, num_layers)
        self.output2_layer = ExpertNetwork(hidden_dim, hidden_dim, num_layers)

        self.out_dense_1 = nn.Linear(hidden_dim, 1)
        self.out_dense_2 = nn.Linear(hidden_dim, 1)
        # 出力層だけGPレイヤーを組み込んでみる
        # self.output1_GP = ToyDeepGPHiddenLayer(
        #     input_dims=hidden_dim,
        #     output_dims=None,
        #     mean_type='constant',
        # )
        # self.output2_GP = ToyDeepGPHiddenLayer(
        #     input_dims=hidden_dim,
        #     output_dims=None,
        #     mean_type='constant',
        # )

        # self.likelihood = GaussianLikelihood()

    def forward(self, x):
        x = F.leaky_relu(self.input_layer(x), negative_slope=0.2)

        expert_outputs = [expert(x).unsqueeze(-1) for expert in self.experts]
        expert_outputs = torch.cat(expert_outputs, dim=-1)

        task1_gate_output = F.softmax(self.task1_gate_layer(x), dim=-1).unsqueeze(1)
        task2_gate_output = F.softmax(self.task2_gate_layer(x), dim=-1).unsqueeze(1)

        weighted_output1 = expert_outputs * task1_gate_output
        weighted_output2 = expert_outputs * task2_gate_output

        output1 = weighted_output1.sum(dim=-1)
        output2 = weighted_output2.sum(dim=-1)

        output1 = self.output1_layer(output1)
        output2 = self.output2_layer(output2)

        output_data_1 = self.out_dense_1(output1)
        output_data_2 = self.out_dense_2(output2)
        # 出力をGPに変えてみる
        # output_data_1 = self.output1_GP(output1)
        # output_data_2 = self.output2_GP(output2)

        return output_data_1, output_data_2

    # def predict(self, x):
    #     output1, output2 = self(x)
    #     mean1, variance1 = output1.mean, output1.variance
    #     mean2, variance2 = output2.mean, output2.variance
    #     return (mean1, variance1), (mean2, variance2)

# モデルの例
input_dim = 101
hidden_dim = 16
experts = 3
num_layers = 5

model = BayesianNuclearDeep(input_dim, hidden_dim, experts, num_layers)

## データ読み込みと訓練用の関数

In [ ]:
dat = Data_Loader(separate_char=',')
# 訓練データ読み込み(とりあえず全部)
train_data_path = '/content/drive/MyDrive/MMOE/dataset/new_df_theta.csv'
valid_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_theta_categorize_0.05-1.csv'
test_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_theta_categorize_0.05-1.csv'
# 物理
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/df_physics_theta_LNIRT_overt0.csv'
# valid_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_physics_UEC2023.csv'
# test_data_path = '/content/drive/MyDrive/MMOE/dataset/test_df_physics_UEC2023.csv'

# UEC2024　物理
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/物理_trainData_overt0.csv'
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/物理_trainData_overt0_theta-LNIRT.csv'
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/物理_trainData_overt0_theta-IRTPRO.csv'
# valid_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_物理.csv'
# test_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_物理.csv'

# UEC2024　数学
# train_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/数学_trainData_overt0.csv'
# valid_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_数学.csv'
# test_data_path = '/content/drive/MyDrive/MMOE/dataset/UEC検定2024/test_data_数学.csv'

# 時間データのみ正規化?
itemsize = 100
# itemsize = 134
# itemsize = 238
# itemsize = 150

train_itemvec, train_response, train_time, train_theta = dat.load_data(train_data_path,mode = "train", itemSize=itemsize)
print(train_time)
print(train_theta)
# print(dat.mean_list, dat.std_list)
# print(train_theta)
train_norm_list = [dat.mean_list, dat.std_list]
print(len(train_norm_list), train_norm_list)
valid_itemvec, valid_response, valid_time, valid_theta = dat.load_data(valid_data_path,mode = "valid", itemSize=itemsize)
test_itemvec, test_response, test_time, test_theta = dat.load_data(test_data_path,mode = "train", itemSize=itemsize)
print(dat.mean_list, dat.std_list)
test_norm_list = [dat.mean_list] + [dat.std_list]
# print(len(train_itemvec))
# print(train_response)
# print(train_time)


label_keys = [f'item{i}' for i in range(itemsize)] + ["response", "time", "student_theta"]
# print(label_keys)
label_values_train = [np.array(train_itemvec).T[i] for i in range(itemsize)] + [train_response, train_time, train_theta]
label_values_valid = [np.array(valid_itemvec).T[i] for i in range(itemsize)] + [valid_response, valid_time, valid_theta]
label_values_test = [np.array(test_itemvec).T[i] for i in range(itemsize)] + [test_response, test_time, test_theta]
# print(label_values)
# ラベルを辞書に変換
data_dict_train = dict(zip(label_keys, label_values_train))
data_dict_valid = dict(zip(label_keys, label_values_valid))
data_dict_test = dict(zip(label_keys, label_values_test))

from torch.utils.data import Dataset, DataLoader
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]


label_keys = [f'item{i}' for i in range(itemsize)]
label_target = ["response", "time"]
# print(label_keys)
label_keys_values_train = np.array([np.array(train_itemvec).T[i] for i in range(itemsize)]+[train_theta]).T
label_target_values_train = np.array([train_response, train_time]).T

# test
label_keys_values_test = np.array([np.array(test_itemvec).T[i] for i in range(itemsize)]+[test_theta]).T
label_target_values_test = np.array([test_response, test_time]).T

# ラベルを辞書に変換
X_train = label_keys_values_train # dict(zip(label_keys, label_keys_values_train))
y_train = label_target_values_train #dict(zip(label_target, label_target_values_train))
train_dataset = CustomDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# testも同様に変換
X_test = label_keys_values_test # dict(zip(label_keys, label_keys_values_test))
y_test = label_target_values_test #dict(zip(label_target, label_target_values_test))
test_dataset = CustomDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# モデルの例
# input_dim = 101
input_dim = itemsize+1
hidden_dim = 16
experts = 3
num_layers = 5

model = BayesianNuclearDeep(input_dim, hidden_dim, experts, num_layers)

def relu(x,epsilon = 0.100):
  return tf.nn.relu(x)+ epsilon

def MSE(targets, estimated_distribution,targets_normalize,error_value,targets_normalize_1): #（标签值，预测值，标签值标准化，？）

  error = targets - estimated_distribution  #标签值-预测值
  bigger = error**2 * (1+(tf.cast(relu(targets_normalize),tf.float32)))
  smaller = error**2 * 0.005
  classification = targets_normalize > 0.7

  loss = tf.where(classification,bigger,smaller)
  Loss_func = tf.reduce_mean(loss, axis=None, keepdims=False, name=None)
  return Loss_func

batch_size=32
data_loader = train_loader #make_dataset(0, batch_size=batch_size)
# print(data_loader)
# model = DeepGPModel()
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = model.to(device)

num_epochs = 10
# mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, len(data_loader.dataset), beta=0.01))

def train_model(net, train_dataset, train_pre_loader, epochs=90, hypara=0.3):
    global retrain
    file_name = 'test'
    alpha = hypara

    mse_loss = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
    # loss関数の推移をみる
    loss_list = []
    # mll = DeepApproximateMLL(VariationalELBO(net.likelihood, model, len(train_pre_loader.dataset), beta=0.01))

    for epoch in tqdm(range(epochs)):
      # データをシャッフルして分割
        # train_indices = torch.randperm(len(train_dataset.dataset))
        # train_dataset = Subset(train_dataset.dataset, train_indices)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        # ここでやらないとダメ?
        # mll = DeepApproximateMLL(VariationalELBO(net.likelihood, model, len(train_loader.dataset), beta=0.01))
        for x_batch, y_batch in train_loader:
          x_batch = x_batch.to(device)
          y_batch = y_batch.to(device)
          # print(y_batch.shape)
          optimizer.zero_grad()
          output = model(x_batch)
          output_ex_1,output_ex_2 = output
          # print(output_ex_1)
          # print(y_batch[0])
          # print(y_batch.T[0].shape)

          label_ex = (y_batch.T[0]).unsqueeze(1)
          error_value_ex = (y_batch.T[1]).unsqueeze(1)
          loss = mse_loss(label_ex, output_ex_1)
          error_loss = mse_loss(error_value_ex, output_ex_2)

          # GPを使った場合
          # loss = -mll(output_ex_1, label_ex)
          # error_loss = -mll(output_ex_2, error_value_ex)

          total_loss = alpha * loss + (1 - alpha) * error_loss
          loss_list.append(total_loss.item())

          total_loss.backward()
          # 重みの値を確認
          # print(net.input_layer.weight)
          # print(net.input_layer.bias)
          # print(net.experts[0].weight)
          # print(net.experts[0].bias)
          # print(net.experts[1].weight)
          # print(net.experts[1].bias)
          # print(net.experts[2].weight)
          # print(net.experts[2].bias)
          # print(net.task1_gate_layer.weight)
          # print(net.task2_gate_layer.weight)
          optimizer.step()

    # loss関数の推移をプロット
    plt.plot(loss_list)
    plt.show()
###########################################################

## 学習と結果確認

In [ ]:
train_model(model, train_dataset, test_loader, epochs=1000)

# テストデータに対する評価
x_true = test_loader.dataset.features
y_true_response = test_loader.dataset.labels.T[0]
y_true_time = test_loader.dataset.labels.T[1]
x_true = x_true.to(device)

model.eval()
predictions_time = []
predictions_response = []
# actuals_time = []
# actuals_response = []
with torch.no_grad():
    # for inputs, targets in test_loader:
    y_response, y_time = model(x_true)
    # predictions_response.extend(y_response.numpy())
    # predictions_time.extend(y_time.numpy())
    # actuals.extend(targets.numpy())
x_true = x_true.to('cpu')
y_response_mean = y_response.to('cpu')#.mean(0)
# y_response_var = y_response.to('cpu')#.mean(0)
# y_response_std = np.sqrt(y_response_var)
y_time_mean = y_time.to('cpu')#.mean(0)
# y_time_var = y_time.to('cpu')#.mean(0)
min = 0
max = 1000
y_time_mean = y_time_mean*(max-min) + min
# y_time_var = y_time_var*(max-min) + min
y_true_time = y_true_time*(max-min) + min

# print(y_time_mean.shape)
# print(y_true_time.shape)
# print(y_time_mean)
# print(y_response_mean)
# print(y_response_std)
# print(y_time_mean)
# print(y_time_std


# csvに保存
# 時間データの標準化から元に戻す
import pandas as pd
import datetime

def load_date():
    now = datetime.datetime.now()
    date = now.strftime('%Y%m%d')
    return date

# print(y_true_response.shape)
# print(y_true_time.shape)
# print(y_response_mean.shape)
# print(y_time_mean.shape)
# print(y_true_response.T[0].shape)
# print(y_true_time.T[0].shape)
data = {'Pred Response': y_response_mean.T[0],
        'Pred Response Time': y_time_mean.T[0],
        'Mean Response': y_true_response,
        'Mean Response Time': y_true_time}

df = pd.DataFrame(data)

date = load_date()
result_path = 'result_physics_LNRT_MTDNN+theta_alpha0.3_' + date + '.csv'

df.to_csv('/content/drive/MyDrive/MMOE/Result/' + result_path, index=False)

# x_item = np.arange(0, 500, 1)
x_item = np.arange(0, itemsize*5, 1)
# グラフの描画
plt.figure(figsize=(8,8))
plt.subplot(411)
plt.plot(x_item, y_true_response, label='True Function')
plt.plot(x_item, y_response_mean, label='Mean Prediction')
# plt.fill_between(x_item.flatten(), y_response_mean.flatten() - y_response_std.flatten(), y_response_mean.flatten() + y_response_std.flatten(), alpha=0.3, label='Uncertainty')
plt.xlabel('x')
plt.ylabel('response')
plt.legend()

# plt.subplot(412)
# plt.plot(x_item, y_response_std)
# plt.xlabel('x')
# plt.ylabel('Std')

plt.subplot(413)
plt.plot(x_item, y_true_time, label='True Time')
plt.plot(x_item, y_time_mean, label='Pred Time')
# plt.fill_between(x_item.flatten(), y_time_mean.flatten() - y_time_std.flatten(), y_time_mean.flatten() + y_time_std.flatten(), alpha=0.3, label='Uncertainty')
plt.xlabel('x')
plt.ylabel('time')
plt.legend()

# plt.subplot(414)
# plt.plot(x_item, y_time_std)
# plt.xlabel('x')
# plt.ylabel('Std')

plt.tight_layout()
plt.show()

# Multi-Task DNN as Gaussian Process

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ExpertNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(hidden_dim, hidden_dim) for _ in range(num_layers)])

    def forward(self, x):
        for layer in self.layers:
            x = F.leaky_relu(layer(x), negative_slope=0.2)
        return x

class BayesianNuclearDeep(DeepGP):#(nn.Module):
    def __init__(self, input_dim, hidden_dim, experts, num_layers):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)

        self.experts = nn.ModuleList([ExpertNetwork(hidden_dim, hidden_dim, num_layers) for _ in range(experts)])

        self.task1_gate_layer = nn.Linear(hidden_dim, experts)
        self.task2_gate_layer = nn.Linear(hidden_dim, experts)

        self.output1_layer = ExpertNetwork(hidden_dim, hidden_dim, num_layers)
        self.output2_layer = ExpertNetwork(hidden_dim, hidden_dim, num_layers)

        # self.out_dense_1 = nn.Linear(hidden_dim, 1)
        # self.out_dense_2 = nn.Linear(hidden_dim, 1)
        # 出力層だけGPレイヤーを組み込んでみる
        self.output1_GP = ToyDeepGPHiddenLayer(
            input_dims=hidden_dim,
            output_dims=None,
            mean_type='constant',
        )
        self.output2_GP = ToyDeepGPHiddenLayer(
            input_dims=hidden_dim,
            output_dims=None,
            mean_type='constant',
        )

        self.likelihood = GaussianLikelihood()

    def forward(self, x):
        x = F.leaky_relu(self.input_layer(x), negative_slope=0.2)

        expert_outputs = [expert(x).unsqueeze(-1) for expert in self.experts]
        expert_outputs = torch.cat(expert_outputs, dim=-1)

        task1_gate_output = F.softmax(self.task1_gate_layer(x), dim=-1).unsqueeze(1)
        task2_gate_output = F.softmax(self.task2_gate_layer(x), dim=-1).unsqueeze(1)

        weighted_output1 = expert_outputs * task1_gate_output
        weighted_output2 = expert_outputs * task2_gate_output

        output1 = weighted_output1.sum(dim=-1)
        output2 = weighted_output2.sum(dim=-1)

        output1 = self.output1_layer(output1)
        output2 = self.output2_layer(output2)

        # output_data_1 = self.out_dense_1(output1)
        # output_data_2 = self.out_dense_2(output2)
        # 出力をGPに変えてみる
        output_data_1 = self.output1_GP(output1)
        output_data_2 = self.output2_GP(output2)

        return output_data_1, output_data_2

    def predict(self, x):
        output1, output2 = self(x)
        mean1, variance1 = output1.mean, output1.variance
        mean2, variance2 = output2.mean, output2.variance
        return (mean1, variance1), (mean2, variance2)

# モデルの例
input_dim = 284
hidden_dim = 16
experts = 3
num_layers = 5

model = BayesianNuclearDeep(input_dim, hidden_dim, experts, num_layers)


## 訓練

In [11]:
def relu(x,epsilon = 0.100):
  return tf.nn.relu(x)+ epsilon

def MSE(targets, estimated_distribution,targets_normalize,error_value,targets_normalize_1): #（标签值，预测值，标签值标准化，？）

  error = targets - estimated_distribution  #标签值-预测值
  bigger = error**2 * (1+(tf.cast(relu(targets_normalize),tf.float32)))
  smaller = error**2 * 0.005
  classification = targets_normalize > 0.7

  loss = tf.where(classification,bigger,smaller)
  Loss_func = tf.reduce_mean(loss, axis=None, keepdims=False, name=None)
  return Loss_func

batch_size=32
data_loader = train_loader #make_dataset(0, batch_size=batch_size)
# print(data_loader)
# model = DeepGPModel()
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = model.to(device)

num_epochs = 10
mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, len(data_loader.dataset), beta=0.01))

def train_model(net, train_dataset, train_pre_loader, epochs=90, hypara=0.3):
    global retrain
    file_name = 'test'
    alpha = hypara

    mse_loss = nn.MSELoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
    # loss関数の推移をみる
    loss_list = []
    # mll = DeepApproximateMLL(VariationalELBO(net.likelihood, model, len(train_pre_loader.dataset), beta=0.01))

    for epoch in tqdm(range(epochs)):
      # データをシャッフルして分割
        # train_indices = torch.randperm(len(train_dataset.dataset))
        # train_dataset = Subset(train_dataset.dataset, train_indices)
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        # ここでやらないとダメ?
        mll = DeepApproximateMLL(VariationalELBO(net.likelihood, model, len(train_loader.dataset), beta=0.01))
        for x_batch, y_batch in train_loader:
          x_batch = x_batch.to(device)
          y_batch = y_batch.to(device)
          # print(y_batch.shape)
          optimizer.zero_grad()
          output = model(x_batch)
          output_ex_1,output_ex_2 = output
          # print(output_ex_1)
          # print(y_batch[0])
          # print(y_batch.T[0].shape)

          label_ex = (y_batch.T[0])#.unsqueeze(1)
          error_value_ex = (y_batch.T[1])#.unsqueeze(1)
          # loss = mse_loss(label_ex, output_ex_1)
          # error_loss = mse_loss(error_value_ex, output_ex_2)

          # GPを使った場合
          loss = -mll(output_ex_1, label_ex)
          error_loss = -mll(output_ex_2, error_value_ex)

          total_loss = alpha * loss + (1 - alpha) * error_loss
          loss_list.append(total_loss.item())

          total_loss.backward()
          # 重みの値を確認
          # print(net.input_layer.weight)
          # print(net.input_layer.bias)
          # print(net.experts[0].weight)
          # print(net.experts[0].bias)
          # print(net.experts[1].weight)
          # print(net.experts[1].bias)
          # print(net.experts[2].weight)
          # print(net.experts[2].bias)
          # print(net.task1_gate_layer.weight)
          # print(net.task2_gate_layer.weight)
          optimizer.step()

    # loss関数の推移をプロット
    plt.plot(loss_list)
    plt.show()

# train_model(model, train_loader, test_loader, epochs=200)

## 学習+結果出力

In [12]:
# モデルを構築
# input_dim = 101
input_dim = itemsize+1
hidden_dim = 16
experts = 3
num_layers = 5

model = BayesianNuclearDeep(input_dim, hidden_dim, experts, num_layers)

# 学習
print("begin training\n")
batch_size=32
# data_loader = train_loader #make_dataset(0, batch_size=batch_size)
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
model = model.to(device)
train_model(model, train_dataset, test_loader, epochs=500, hypara=0.3)
# 予測結果を表示
x_true = test_loader.dataset.features
y_true = test_loader.dataset.labels.T[0]
y_true_time = test_loader.dataset.labels.T[1]
# x_true = torch.from_numpy(x_true).float().view(-1, 1)

model.eval()
x_true = x_true.to(device)
y_preds = []
# with torch.no_grad():
#     y_mean, y_var = model(x_true)

# x_true = x_true.cpu()
# y_mean = y_mean.cpu()
# y_var = y_var.cpu()

# GPを追加した場合
with torch.no_grad():
    y_response, y_time = model.predict(x_true)
# print(y_response)
# print(y_time.shape)
x_true = x_true.to('cpu')
y_response_mean = y_response[0].to('cpu').mean(0)
y_response_var = y_response[1].to('cpu').mean(0)
y_response_std = np.sqrt(y_response_var)
y_time_mean = y_time[0].to('cpu').mean(0)
y_time_var = y_time[1].to('cpu').mean(0)
y_time_std = np.sqrt(y_time_var)
# print(y_time_mean)
min = 0
max = 1000
y_time_mean = y_time_mean*(max-min) + min
y_time_var = y_time_var*(max-min) + min
y_true_time = y_true_time*(max-min) + min
y_time_std = np.sqrt(y_time_var)

# csvに保存
# 時間データの標準化から元に戻す
import pandas as pd
import datetime

def load_date():
    now = datetime.datetime.now()
    date = now.strftime('%Y%m%d')
    return date

data = {'Pred Response': y_response_mean,
        'Pred Response Var' : y_response_var,
        'Pred Response Time': y_time_mean,
        'Pred Response Time Var' : y_time_var,
        'Mean Response': y_true,
        'Mean Response Time': y_true_time}

df = pd.DataFrame(data)

date = load_date()
result_path = 'result_information_GP_LNRT+theta_alpha0.3_' + date + '.csv'

df.to_csv('/content/drive/MyDrive/MMOE/Result/' + result_path, index=False)

# x_item = np.arange(0, 500, 1)
x_item = np.arange(0, itemsize*5, 1)
# グラフの描画
plt.figure(figsize=(8,8))
plt.subplot(411)
plt.plot(x_item, y_true, label='True Function')
plt.plot(x_item, y_response_mean, label='Mean Prediction')
plt.fill_between(x_item.flatten(), y_response_mean.flatten() - y_response_std.flatten(), y_response_mean.flatten() + y_response_std.flatten(), alpha=0.3, label='Uncertainty')
plt.xlabel('x')
plt.ylabel('response')
plt.legend()

plt.subplot(412)
plt.plot(x_item, y_response_std)
plt.xlabel('x')
plt.ylabel('Std')

plt.subplot(413)
plt.plot(x_item, y_true_time, label='True Time')
plt.plot(x_item, y_time_mean, label='Pred Time')
plt.fill_between(x_item.flatten(), y_time_mean.flatten() - y_time_std.flatten(), y_time_mean.flatten() + y_time_std.flatten(), alpha=0.3, label='Uncertainty')
plt.xlabel('x')
plt.ylabel('time')
plt.legend()

plt.subplot(414)
plt.plot(x_item, y_time_std)
plt.xlabel('x')
plt.ylabel('Std')

plt.tight_layout()
plt.show()


begin training



  1%|          | 3/500 [01:22<3:47:49, 27.50s/it]


KeyboardInterrupt: 

## αチューニング用

In [ ]:
import pandas as pd
import datetime
def load_date():
      now = datetime.datetime.now()
      date = now.strftime('%Y%m%d')
      return date

def research_alpha(model, train_ds, train_loader, test_loader,output_path, alpha):
  train_model(model, train_ds, train_loader, epochs=100, hypara = alpha)
  # 予測結果を表示
  x_true = test_loader.dataset.features
  y_true = test_loader.dataset.labels.T[0]
  y_true_time = test_loader.dataset.labels.T[1]
  # x_true = torch.from_numpy(x_true).float().view(-1, 1)

  model.eval()
  x_true = x_true.to(device)
  y_preds = []
  # with torch.no_grad():
  #     y_mean, y_var = model(x_true)

  # x_true = x_true.cpu()
  # y_mean = y_mean.cpu()
  # y_var = y_var.cpu()

  # GPを追加した場合
  with torch.no_grad():
      y_response, y_time = model.predict(x_true)
  # print(y_response)
  # print(y_time.shape)
  x_true = x_true.to('cpu')
  y_response_mean = y_response[0].to('cpu').mean(0)
  y_response_var = y_response[1].to('cpu').mean(0)
  y_response_std = np.sqrt(y_response_var)
  y_time_mean = y_time[0].to('cpu').mean(0)
  y_time_var = y_time[1].to('cpu').mean(0)
  y_time_std = np.sqrt(y_time_var)
  # print(y_time_mean)
  min = 0
  max = 1000
  y_time_mean = y_time_mean*(max-min) + min
  y_time_var = y_time_var*(max-min) + min
  y_true_time = y_true_time*(max-min) + min
  y_time_std = np.sqrt(y_time_var)

  print(y_time_mean.shape)
  # responseとtimeのRMSEを計算
  # y_response_mean = y_response_mean.cpu().numpy()
  # y_response_var = y_response_var.cpu().numpy()
  # y_time_mean = y_time_mean.cpu().numpy()
  # y_time_var = y_time_var.cpu().numpy()
  # RMSE_response = np.sqrt(np.mean((y_true - y_response_mean)**2))
  # RMSE_time = np.sqrt(np.mean((y_true_time - y_time_mean)**2))
  # print("RMSE_response", RMSE_response)
  # print("RMSE_time", RMSE_time)

  # csvに保存
  # 時間データの標準化から元に戻す

  data = {'Pred Response': y_response_mean,
          'Pred Response Var' : y_response_var,
          'Pred Response Time': y_time_mean,
          'Pred Response Time Var' : y_time_var,
          'Mean Response': y_true,
          'Mean Response Time': y_true_time}

  df = pd.DataFrame(data)

  # dfの要素からRMSEを計算
  RMSE_response = np.sqrt(np.mean((df["Pred Response"] - df["Mean Response"])**2))
  RMSE_time = np.sqrt(np.mean((df["Pred Response Time"] - df["Mean Response Time"])**2))
  print("alpha", alpha)
  print("RMSE_response", RMSE_response)
  print("RMSE_time", RMSE_time)
  print("--")

  # date = load_date()
  # result_path = 'result_GP+theta_' + output_path + "_" + date + '.csv'

  # df.to_csv('/content/drive/MyDrive/MMOE/Result/' + result_path, index=False)

In [ ]:
train_paths = ["/content/drive/MyDrive/MMOE/dataset/CrossValid_data/uec_math_time_overt0_train_" + str(i) + ".csv" for i in range(10)]
test_paths = ["/content/drive/MyDrive/MMOE/dataset/CrossValid_data/uec_math_time_overt0_test_" + str(i) + ".csv" for i in range(10)]
for k in range(1, 9):
  train_data_path = train_paths[k]
  test_data_path = test_paths[k]
  # 時間データのみ正規化?
  train_itemvec, train_response, train_time, train_theta = dat.load_data(train_data_path,mode = "train")
  # print(dat.mean_list, dat.std_list)
  # print(train_theta)
  train_norm_list = [dat.mean_list, dat.std_list]
  print(len(train_norm_list), train_norm_list)
  # valid_itemvec, valid_response, valid_time, valid_theta = dat.load_data(valid_data_path,mode = "valid")
  test_itemvec, test_response, test_time, test_theta = dat.load_data(test_data_path,mode = "train")
  print(dat.mean_list, dat.std_list)
  test_norm_list = [dat.mean_list] + [dat.std_list]
  # print(len(train_itemvec))
  # print(train_response)
  # print(train_time)
  label_keys = [f'item{i}' for i in range(100)] + ["response", "time", "student_theta"]
  # print(label_keys)
  label_values_train = [np.array(train_itemvec).T[i] for i in range(100)] + [train_response, train_time, train_theta]
  label_values_valid = [np.array(valid_itemvec).T[i] for i in range(100)] + [valid_response, valid_time, valid_theta]
  label_values_test = [np.array(test_itemvec).T[i] for i in range(100)] + [test_response, test_time, test_theta]
  # print(label_values)
  # ラベルを辞書に変換
  data_dict_train = dict(zip(label_keys, label_values_train))
  data_dict_valid = dict(zip(label_keys, label_values_valid))
  data_dict_test = dict(zip(label_keys, label_values_test))

  from torch.utils.data import Dataset, DataLoader
  class CustomDataset(Dataset):
      def __init__(self, features, labels):
          self.features = torch.tensor(features, dtype=torch.float32)
          self.labels = torch.tensor(labels, dtype=torch.float32)

      def __len__(self):
          return len(self.features)

      def __getitem__(self, idx):
          return self.features[idx], self.labels[idx]


  label_keys = [f'item{i}' for i in range(100)]
  label_target = ["response", "time"]
  # print(label_keys)
  label_keys_values_train = np.array([np.array(train_itemvec).T[i] for i in range(100)]+[train_theta]).T
  label_target_values_train = np.array([train_response, train_time]).T

  # test
  label_keys_values_test = np.array([np.array(test_itemvec).T[i] for i in range(100)]+[test_theta]).T
  label_target_values_test = np.array([test_response, test_time]).T

  # ラベルを辞書に変換
  X_train = label_keys_values_train # dict(zip(label_keys, label_keys_values_train))
  y_train = label_target_values_train #dict(zip(label_target, label_target_values_train))
  train_dataset = CustomDataset(X_train, y_train)
  train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

  # testも同様に変換
  X_test = label_keys_values_test # dict(zip(label_keys, label_keys_values_test))
  y_test = label_target_values_test #dict(zip(label_target, label_target_values_test))
  test_dataset = CustomDataset(X_test, y_test)
  test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

  for i in range(1, 9):
    alpha = 0.1*i
    output_path = "alpha" + str(alpha)

    # モデルを構築
    input_dim = 101
    hidden_dim = 16
    experts = 3
    num_layers = 5

    model = BayesianNuclearDeep(input_dim, hidden_dim, experts, num_layers)

    # 学習
    print("begin training\n")
    batch_size=32
    # data_loader = train_loader #make_dataset(0, batch_size=batch_size)
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    model = model.to(device)
    # mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, len(data_loader.dataset), beta=0.01))
    research_alpha(model, train_dataset, train_loader, test_loader, output_path, alpha=alpha)

2 [0, 1000]
0 1000
begin training



100%|██████████| 100/100 [27:02<00:00, 16.22s/it]


torch.Size([1951])
alpha 0.1
RMSE_response 0.37902683
RMSE_time 106.61883
--
begin training



100%|██████████| 100/100 [26:50<00:00, 16.11s/it]


torch.Size([1951])
alpha 0.2
RMSE_response 0.3766741
RMSE_time 107.30601
--
begin training



...

torch.Size([1951])
alpha 0.2
RMSE_response 0.38586423
RMSE_time 107.41191
--
begin training



100%|██████████| 100/100 [27:36<00:00, 16.56s/it]


torch.Size([1951])
alpha 0.30000000000000004
RMSE_response 0.38416708
RMSE_time 107.7699
--
begin training



 10%|█         | 10/100 [02:55<26:23, 17.59s/it]


KeyboardInterrupt: 

In [ ]:
print(train_dataset.features.shape)
print(train_dataset.labels.shape)
print(len(train_dataset.features))

torch.Size([17559, 100])
torch.Size([17559, 2])
17559


## 交差検証による予測精度の評価

In [ ]:
# 岸田さんのクロスバリデーション用データを、こっちの使用に合わせて成形する
timeData_path = '/content/drive/MyDrive/MMOE/dataset/Kishida_CrossValidation/uec_math_timeovert0'
correctData_path = '/content/drive/MyDrive/MMOE/dataset/Kishida_CrossValidation/uecmath_time_overt0_0315'
studentID_itemID_path  = '/content/drive/MyDrive/MMOE/dataset/new_df_theta.csv'

for k in range(10):
  timeData = pd.read_csv(timeData_path + "/uecmath_time_overt0_train_" + str(k) + ".csv", header=None)
  correctData = pd.read_csv(correctData_path +  "/uecmath_correct_overt0_train_" +str(k) + ".csv", header=None)
  studentID_itemID = pd.read_csv(studentID_itemID_path)
  # print(timeData.shape)
  # print(correctData.shape)
  # ヘッダーを設定
  timeData.columns =[f'item{i}' for i in range(100)]
  correctData.columns = [f'item{i}' for i in range(100)]
  import math
  import numpy as np

  maked_data = []
  for i in range(timeData.shape[0]):
    student_list = []
    for j in range(timeData.shape[1]):

      if timeData.iloc[i,j] > 0 and (np.isnan(correctData.iloc[i, j])==False):
        # j番目を1とするone-hotベクトルを作成
        one_hot_vector = np.zeros(100)
        one_hot_vector[j] = 1
        list_user_item = [i, j] + list(one_hot_vector) + [correctData.iloc[i,j], timeData.iloc[i,j]] # + ["student_theta"]
        maked_data.append(list_user_item)

        student_list.append(timeDB.iloc[i,j])

      elif timeData.iloc[i,j] > 0 and (np.isnan(correctData.iloc[i, j])==True):
        # j番目を1とするone-hotベクトルを作成
        one_hot_vector = np.zeros(100)
        one_hot_vector[j] = 1
        list_user_item = [i, j] + list(one_hot_vector) + [0, timeData.iloc[i,j]]
        maked_data.append(list_user_item)

        student_list.append(timeDB.iloc[i,j])

      # 受験者iのstudentIDを判断
      # 2つのリストの中身が一致する場合
      for k in range(timeData.shape[0]):
        print(studentID_itemID[k][:-1])
        studentID_itemID["time"][k] = int(studentID_itemID["time"][k])
        if list(studentID_itemID[k][:-1]) == student_list:
          student_list = student_list + [studentID_itemID["student_theta"][k]]
        # else:
          # student_list = student_list + [



  maked_data = np.array(maked_data)
  # print(maked_data.shape)
  # print(maked_data)
  # dfに変換
  df = pd.DataFrame(maked_data)
  # ヘッダーを設定
  df.columns = ["user", "item"] + [f'item{i}' for i in range(100)] + ["response", "time"]
  # csvファイルに保存
  df.to_csv('/content/drive/MyDrive/MMOE/dataset/Kishida_CrossValidation/uec_math_for_MultiTaskDNN/uecmath_time_overt0_train_' + str(k) + '.csv', index=False)

KeyError: 0

In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error
import numpy as np
import csv

def CrossValid_eval():

  # パスのリスト（これは仮の例です）
  train_paths = ["/content/drive/MyDrive/MMOE/dataset/CrossValid_data/uec_math_time_overt0_train_" + str(i) + ".csv" for i in range(10)]
  test_paths = ["/content/drive/MyDrive/MMOE/dataset/CrossValid_data/uec_math_time_overt0_test_" + str(i) + ".csv" for i in range(10)]

  # 交差検証のループ
  mse_response_scores = []
  mse_time_scores = []
  rmse_response_scores = []
  rmse_time_scores = []
  output_response_std = []
  output_time_std = []

  for train_path, test_path in zip(train_paths, test_paths):

      # データの読み込み
      dat = Data_Loader(separate_char=',')
      train_itemvec, train_response, train_time, train_theta = dat.load_data(train_path,mode = "train")
      train_norm_list = [dat.mean_list] + [dat.std_list]
      test_itemvec, test_response, test_time, test_theta = dat.load_data(test_path,mode = "train")
      test_norm_list = [dat.mean_list] + [dat.std_list]
      label_keys = [f'item{i}' for i in range(100)] + ["response", "time", "student_theta"]
      label_values_train = [np.array(train_itemvec).T[i] for i in range(100)] + [train_response, train_time, train_theta]
      label_values_test = [np.array(test_itemvec).T[i] for i in range(100)] + [test_response, test_time, test_theta]
      # ラベルを辞書に変換
      data_dict_train = dict(zip(label_keys, label_values_train))
      data_dict_test = dict(zip(label_keys, label_values_test))

      # データローダーの作成
      label_keys = [f'item{i}' for i in range(100)] + ["student_theta"]
      label_target = ["response", "time"]
      # print(label_keys)
      label_keys_values_train = np.array([np.array(train_itemvec).T[i] for i in range(100)] + np.array(train_theta).T).T
      label_target_values_train = np.array([train_response, train_time]).T

      # test
      label_keys_values_test = np.array([np.array(test_itemvec).T[i] for i in range(100)]+ [test_theta]).T
      # label_keys_values_testの右側に列を追加
      label_target_values_test = np.array([test_response, test_time]).T

      print(label_keys_values_test.shape)

      # ラベルを辞書に変換
      X_train = label_keys_values_train # dict(zip(label_keys, label_keys_values_train))
      y_train = label_target_values_train #dict(zip(label_target, label_target_values_train))
      train_dataset = CustomDataset(X_train, y_train)
      train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

      # testも同様に変換
      X_test = label_keys_values_test # dict(zip(label_keys, label_keys_values_test))
      y_test = label_target_values_test #dict(zip(label_target, label_target_values_test))
      test_dataset = CustomDataset(X_test, y_test)
      test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

      # モデルを構築
      input_dim = 101
      hidden_dim = 16
      experts = 3
      num_layers = 5

      model = BayesianNuclearDeep(input_dim, hidden_dim, experts, num_layers)

      # 学習
      print("begin training\n")
      batch_size=32
      data_loader = train_loader #make_dataset(0, batch_size=batch_size)
      device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
      model = model.to(device)
      mll = DeepApproximateMLL(VariationalELBO(model.likelihood, model, len(data_loader.dataset), beta=0.01))

      train_model(model, train_loader, test_loader, epochs=10)

      # テストデータに対する評価
      x_true = test_loader.dataset.features
      y_true_response = test_loader.dataset.labels.T[0]
      y_true_time = test_loader.dataset.labels.T[1]
      x_true = x_true.to(device)

      model.eval()
      predictions_time = []
      predictions_response = []
      # actuals_time = []
      # actuals_response = []
      with torch.no_grad():
          # for inputs, targets in test_loader:
          y_response, y_time = model.predict(x_true)
          # predictions_response.extend(y_response.numpy())
          # predictions_time.extend(y_time.numpy())
          # actuals.extend(targets.numpy())
      x_true = x_true.to('cpu')
      y_response_mean = y_response[0].to('cpu').mean(0)
      y_response_var = y_response[1].to('cpu').mean(0)
      y_response_std = np.sqrt(y_response_var)
      y_time_mean = y_time[0].to('cpu').mean(0)
      y_time_var = y_time[1].to('cpu').mean(0)
      print(y_time_mean)
      print(y_true_time)
      # y_time_mean = y_time_mean.to('cpu').mean(0
      min = 0
      max = 1000
      y_time_mean = y_time_mean*(max-min) + min
      y_time_var = y_time_var*(max-min) + min
      y_true_time = y_true_time*(max-min) + min
      y_time_std = np.sqrt(y_time_var)

      print(y_time_mean)
      print(y_true_time)

      mse_response = mean_squared_error(y_true_response, y_response_mean)
      print("MSE for response:", mse_response)
      mse_response_scores.append(mse_response)
      mse_time = mean_squared_error(y_true_time, y_time_mean)
      print("MSE for time:", mse_time)
      mse_time_scores.append(mse_time)

      # RMSEも計算
      rmse_response = np.sqrt(mse_response)
      print("RMSE for response:", rmse_response)
      rmse_time = np.sqrt(mse_time)
      print("RMSE for time:", rmse_time)
      rmse_response_scores.append(rmse_response)
      rmse_time_scores.append(rmse_time)

      # 不確かさを足し合わせる
      output_response_std.append(y_response_std)
      output_time_std.append(y_time_std)


  # 交差検証の結果を表示
  print("MSE scores for each fold:", mse_response_scores)
  print("Average MSE:", np.mean(mse_response_scores))
  print("Std MSE:", np.std(mse_response_scores))
  print("MSE scores for each fold:", mse_time_scores)
  print("Average MSE:", np.mean(mse_time_scores))
  print("Std MSE:", np.std(mse_time_scores))
  print("RMSE scores for each fold:", rmse_response_scores)
  print("Average RMSE:", np.mean(rmse_response_scores))
  print("Std RMSE:", np.std(rmse_response_scores))
  print("RMSE scores for each fold:", rmse_time_scores)
  print("Average RMSE:", np.mean(rmse_time_scores))
  print("Std RMSE:", np.std(rmse_time_scores))

  # 結果をcsvに保存
  filename = "result_MultiTaskDNNGP_itemvec_theta.csv"
  with open("/content/drive/MyDrive/MMOE/dataset/CrossValid_data/result/" + filename, "w") as f:
      writer = csv.writer(f)
      writer.writerow(["MSE", "RMSE"])
      writer.writerow([np.mean(mse_response_scores), np.mean(rmse_response_scores)])
      writer.writerow([np.mean(mse_time_scores), np.mean(rmse_time_scores)])
      writer.writerow(["Average", "Average"])
      writer.writerow([np.std(mse_response_scores), np.std(rmse_response_scores)])
      writer.writerow([np.std(mse_time_scores), np.std(rmse_time_scores)])
      writer.writerow(["Std", "Std"])
      writer.writerow([np.mean(mse_response_scores), np.mean(rmse_response_scores)])
      writer.writerow([np.mean(mse_time_scores), np.mean(rmse_time_scores)])




In [ ]:
CrossValid_eval()

(1952, 101)
begin training



  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x100 and 101x16)